# 爆破用

In [ ]:
# @markdown #💣削除用！注意！  
# @markdown 例 : `/content/test`で`/content/test`を中身ごと削除  
import shutil
dir = "/content/test" # @param{type:"string"}
shutil.rmtree(dir)

In [ ]:
%cd /content/

# Create repo, Download, Upload to 🤗

In [ ]:
# git lfs initializer
# @markdown <hr>

# @markdown ##installing requirements
# @markdown <hr>

# @markdown #🌱Initializer Area
!pip install huggingface_hub
!pip install --upgrade gdown

In [ ]:
# @markdown <hr>

# @markdown ##create a repo as model, dataset or space
# @markdown <hr>

# @markdown #🤗 Creating Repo Area
repo_name = "user_id/repo_id" # @param{type:"string"}
token = "hf_XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX" # @param{type:"string"}

if token:
  hf_token = token

repo_type = "model" # @param["model", "dataset", "space"] {allow-input:true}
if repo_type not in ["model", "dataset", "space"]:
  repo_type = None

  private = True # @param{type:"boolean"}

from huggingface_hub import create_repo, login, HfApi
login(token=token)
create_repo(repo_id=repo_name, repo_type=repo_type, private=private, exist_ok=True)

In [ ]:
# @markdown <hr>

# @markdown ##downloading area (civitai, 🤗, gigafile)
# @markdown <hr>

# @markdown #⬇️Downloading Area

## for regex
import re

## Variables
MODEL_URLS = "https://civitai.com/api/download/models/11657, https://huggingface.co/unkounko/BalloonMix/resolve/main/LoRA/balloon.pt, https://tenor.com/view/%E3%81%A1%E3%81%83%E3%81%8B%E3%82%8F-gif-26614648" # @param {type:"string"}
urls_list = [url.strip() for url in MODEL_URLS.split(',')]

SAVE_AS_CHECK = False # @param {type:"boolean"}
SAVE_AS_FILENAMES = "a.file, b.file, c.file" # @param {type:"string"}
filenames_list = [url.strip() for url in SAVE_AS_FILENAMES.split(',')]

## Download with Gigafile Check
try:
  if SAVE_AS_CHECK:
    assert len(urls_list)==len(filenames_list)
except AssertionError:
  print("Error: MODEL_URLS and SAVE_AS_FILENAMES have different lengths.")
else:
  for i in range(len(urls_list)):
    MODEL_URL = urls_list[i]
    GIGA_CHECK = "gigafile.nu" in MODEL_URL
    GOOGLE_CHECK = "drive.google.com" in MODEL_URL

    if GIGA_CHECK:
      BUTTON_URL = MODEL_URL.replace("gigafile.nu/", "gigafile.nu/download.php?file=")
      GET_GARBAGE = re.search(r"[^/]*$", MODEL_URL).group()
      print(GET_GARBAGE)
      !wget --keep-session-cookies --save-cookies=cookies.txt $MODEL_URL

      if SAVE_AS_CHECK:
        SAVE_AS_FILENAME = filenames_list[i]
        !wget --load-cookies cookies.txt -O $SAVE_AS_FILENAME $BUTTON_URL
      else: 
        !wget --load-cookies cookies.txt $BUTTON_URL --content-disposition

      # clean for gigafile
      !rm ./cookies.txt
      !rm ./{GET_GARBAGE}

    elif GOOGLE_CHECK:
      if "/file/d/" in MODEL_URL:
        drive_id = re.search(r"(?<=/file/d/)[\w-]+", MODEL_URL).group(0)
      else:
        drive_id = re.search(r"(?<=id=)[\w-]+", MODEL_URL).group(0)
        
      !gdown {drive_id}

    else:
      if SAVE_AS_CHECK:
        SAVE_AS_FILENAME = filenames_list[i]
        !wget -O $SAVE_AS_FILENAME $MODEL_URL
      else:
        !wget $MODEL_URL --content-disposition

In [ ]:
# @markdown <hr>

# @markdown ##Uploading your files to the repo
# @markdown <hr>

# @markdown #🤗 Uploading Area
import os

source_path = "/content/testfolder" #@param {type:"string"}
destination_folder = "aabb_01/a" #@param {type:"string"}
ignore_fnmatch = ".txt, .json" #@param {type:"string"}
fnmatch_list = [url.strip() for url in ignore_fnmatch.split(',')]

if os.path.exists(source_path) and os.path.isfile(source_path):
  filename = os.path.basename(source_path)
  if destination_folder:
    if re.search("/$", destination_folder):
      destination_folder = destination_folder + filename
    else:
      destination_folder = destination_folder + "/" + filename

  HfApi().upload_file(
      path_or_fileobj=source_path,
      path_in_repo=destination_folder,
      repo_id=repo_name,
  )

elif os.path.exists(source_path) and os.path.isdir(source_path):
  HfApi().upload_folder(
      folder_path=source_path,
      path_in_repo=destination_folder,
      repo_id=repo_name,
      ignore_patterns=fnmatch_list,
  )

else:
  print("{source_path} is not found")